In [53]:
import pandas as pd
import requests
import re
import time

In [79]:
df = pd.read_csv("products.csv", sep=";")

In [3]:
df.head(10)

,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33 MGA,Ingredients: sugar and milk
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,May contain sugar
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish"
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish"


### On aimerait avoir une colonne de prix unifiés en euros. Problème: la currency n'est pas indiquée pour tous les produits: il va falloir essayer de "deviner" les currency manquantes, en se basant sur l'adresse IP de l'utilisateur.

In [62]:
def get_country_ip(ip, currency="EUR"):
    url = f"http://www.geoplugin.net/json.gp?ip={ip}&base_currency={currency}"
    resp = requests.get(url)
    if resp.status_code != 200:
        print(resp.status_code)
        return 0
    else:
        data = resp.json()
        return data['geoplugin_currencyConverter']

In [9]:
def get_currencies_rate():
    # return all exchange rate based on USD
    url = f"http://www.apilayer.net/api/live?access_key=acdbfc1a4d1cc81cea718c28a71ddd18"
    resp = requests.get(url)
    if resp.status_code != 200:
        return 0
    else:
        return resp.json()['quotes']

In [10]:
currencies_rate = get_currencies_rate()

In [99]:
# ^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])$

In [80]:
def compute_price_EUR(price, ip_address, currencies_rate):
    elmnt = price.split(" ")
    ip_regex = "^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])$"
    if len(elmnt)>1:
        currency = elmnt[1]
        if currency == "EUR":
            return float(elmnt[0])
        else:
            return float(elmnt[0])/(currencies_rate['USD'+currency]/currencies_rate['USDEUR'] )
        
    elif re.search(ip_regex, ip_address) is not None:
        currency_rate = get_country_ip(ip_address)
        # use to avoid to be blacklisted -> the API authorizes 120 queries/second max
        time.sleep(.600)
        if currency_rate !=0:
            return float(price)/currency_rate
        else:
            return 0
    else:
        return 0
    

In [81]:
df['pivot_price']=df.apply(lambda row: compute_price_EUR(row['price'], row['ip_address'], currencies_rate), axis=1)

In [82]:
df

,username,ip_address,product,price,infos,pivot_price
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,0.000000
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,0.000000
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,0.000000
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,314.713284
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",7.865026
...,...,...,...,...,...,...
195,ccannop5f,174.5.73.129,Nantucket Orange Juice,713.6,"Contains gluten, mustard, sugar, milk and fish",488.265481
196,lprovis5g,191.69.45.257,"Jam - Strawberry, 20 Ml Jar",107.78,Contains gluten and sugar,0.000000
197,iollarenshaw5h,206.30.25.226,"Juice - Apple, 341 Ml",447.17 RUB,May contain gluten and soja,6.277302
198,mjablonski5i,44.133.211.182,"Mushroom - Chantrelle, Fresh",632.29,Ingredients: soja and sugar,568.299479


In [83]:
cleaned_data = df.loc[df['pivot_price'] !=0 ]
len(cleaned_data)

159

We cannot manage to find the country currency for 41 purchase

In [69]:
cleaned_data

,username,ip_address,product,price,infos,pivot_price
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,314.713284
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",7.865026
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33 MGA,Ingredients: sugar and milk,0.087496
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,5.242235
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",4.120455
...,...,...,...,...,...,...
191,ecasserley5b,82.78.162.116,Cup - 8oz Coffee Perforated,714.56,Ingredients: sugar and fish,150.297625
193,tliddyard5d,93.22.46.177,Shrimp - Black Tiger 6 - 8,300.03,"Ingredients: sugar, egg and fish",300.030000
195,ccannop5f,174.5.73.129,Nantucket Orange Juice,713.6,"Contains gluten, mustard, sugar, milk and fish",488.265481
197,iollarenshaw5h,206.30.25.226,"Juice - Apple, 341 Ml",447.17 RUB,May contain gluten and soja,6.277302


### La colonne "infos" liste des ingrédients présents dans le produit. On préfèrerait avoir une colonne de type bool par ingrédient, indiquant si le produit contient ou non cet ingrédient.

In [84]:
df.infos.head()

0                  May contain sugar
1           Contains peanut and fish
2      Ingredients: mustard and fish
3                    Contains gluten
4    May contain sugar, egg and fish
Name: infos, dtype: object

In [85]:
# On place chaque mot dans une liste et on enlève les stops words
def split_and_clean_words(ingredients):
    list_ingred  = ingredients.replace(",", "").split(" ")
    stop_words = {"May", "contain", "Contains","Ingredients:", "and"}
    return list(set(list_ingred).difference(stop_words))

# on effectue le traitement pour chaque ligne, la colonne infos est supprimée au profit de infos_list
df['infos_list'] = df.pop('infos').apply(lambda ingredients: split_and_clean_words(ingredients) )

In [86]:
df.head()

,username,ip_address,product,price,pivot_price,infos_list
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,0.000000,[sugar]
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,0.000000,"[peanut, fish]"
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,0.000000,"[fish, mustard]"
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,314.713284,[gluten]
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,7.865026,"[egg, sugar, fish]"


In [87]:
from sklearn.preprocessing import MultiLabelBinarizer

# On importe ce  module de sklearn qui permet d'encoder avec des 1 et des 0 les 
# valeurs contenues dans des listes dans autant de colonnes qu'il y a de valeurs différentes
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('infos_list')),
                          columns=mlb.classes_,
                          index=df.index).replace({1: True, 0: False}))
# la fonction replace permet de remplacer les 1 par True et 0 par False, comme noussouhaitions avoir des 
# colonnes de booléens

In [88]:
df.head()

,username,ip_address,product,price,pivot_price,egg,fish,gluten,milk,mustard,peanut,soja,sugar
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,0.000000,False,False,False,False,False,False,False,True
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,0.000000,False,True,False,False,False,True,False,False
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,0.000000,False,True,False,False,True,False,False,False
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,314.713284,False,False,True,False,False,False,False,False
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,7.865026,True,True,False,False,False,False,False,True
